In [1]:
import torch

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval()  # disable dropout (or leave in train mode to finetune)